# Sallve - Web Scraping

In [ ]:
import sys, os, re, json, time, unicodedata
from urllib.parse import urljoin, urlparse, parse_qs, urlencode
from pathlib import Path

import requests
import pandas as pd
from bs4 import BeautifulSoup

# ========= GLOBAIS FALTANTES (definidos aqui p/ evitar NameError) =========


# ========= IMPORTS DOS MODELS =========
from skin import (
    SKIN_TYPE_CANONICAL_ORDER,
    SKIN_TYPE_SYNONYMS_PT,
)
from exclude import EXCLUDE_KEYWORDS
from ingredient import INGREDIENTES_VALIDOS
from benefits import BENEFIT_SYNONYMS_PT, BENEFIT_CANONICAL_ORDER
from category import CATEGORY_HINTS 

## Configurações Iniciais -> variáveis e requisição

In [ ]:
BASE_URL       = os.environ.get("SALLVE_BASE_URL", "https://www.sallve.com.br")
COLLECTION_URL = os.environ.get("SALLVE_COLLECTION_URL", f"{BASE_URL}/collections/loja")
HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
        "(KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36"
    ),
    "Accept-Language": "pt-BR,pt;q=0.9,en;q=0.8",
}
IMAGES_DIR = Path("images")
IMAGES_DIR.mkdir(parents=True, exist_ok=True)

## Funções Utilitárias

In [ ]:

def _strip_accents_lower(s: str) -> str:
    if not isinstance(s, str): return ""
    s = s.strip().lower()
    s = unicodedata.normalize("NFD", s)
    s = "".join(ch for ch in s if unicodedata.category(ch) != "Mn")
    s = re.sub(r"\s+", " ", s)
    return s

def _safe_join_url(url: str) -> str:
    return "https:" + url if isinstance(url, str) and url.startswith("//") else url

def _tokenize_ingredientes(texto: str):
    if not texto: return []
    texto = texto.replace("\n", " ").replace("\r", " ")
    partes = re.split(r"[;,·•|\u2022]|(?:\s{2,})|,|\.", texto)
    return [p.strip() for p in partes if p and p.strip()]

def _padroniza_por_lista(tokens, lista_validos):
    valid_norm_map = {_strip_accents_lower(v): v for v in lista_validos}
    padronizados, vistos = [], set()
    for tok in tokens:
        n = _strip_accents_lower(tok)
        # normalizações especiais
        if "hialuronato" in n and "sodio" in n and "acido hialuronico" in valid_norm_map:
            key = "acido hialuronico"
        elif "matrixyl" in n:
            key = _strip_accents_lower("peptídeos matrixyl")
        else:
            key = None
            for kn in valid_norm_map.keys():
                if n == kn or n.startswith(kn) or kn in n:
                    key = kn
                    break
        if key and key not in vistos:
            padronizados.append(valid_norm_map[key]); vistos.add(key)
    return padronizados

def _add_or_replace_page_param(url: str, page: int) -> str:
    parsed = urlparse(url)
    q = parse_qs(parsed.query, keep_blank_values=True)
    q["page"] = [str(page)]
    new_query = urlencode({k: v[0] if isinstance(v, list) and len(v) == 1 else v for k, v in q.items()}, doseq=True)
    return parsed._replace(query=new_query).geturl()

def _sanitize_filename(name: str) -> str:
    base = _strip_accents_lower(name)
    base = re.sub(r"[^a-z0-9]+", "-", base).strip("-")
    base = re.sub(r"-{2,}", "-", base)
    return base or "produto"

def _normalize_price_text(txt: str) -> str | None:
    if not txt: return None
    t = re.sub(r"\s+", " ", txt).strip().replace("\xa0", " ")
    m = re.search(r"R\$\s*\d{1,3}(?:\.\d{3})*,\d{2}", t)
    if m:
        p = m.group(0)
        p = re.sub(r"\s+", " ", p).replace("R$ ", "R$ ").strip()
        return p
    m2 = re.search(r"\d{1,3}(?:\.\d{3})*,\d{2}", t)
    if m2:
        return "R$ " + m2.group(0)
    return None

def should_exclude_product(product_name):
    if not product_name: return True
    product_name_lower = product_name.lower()
    return any(k in product_name_lower for k in EXCLUDE_KEYWORDS)


_CATEGORY_ORDER_MAP = {name: i for i, name in enumerate(CATEGORY_CANONICAL_ORDER)}

def classify_category_from_name(name: str, description: str | None = None) -> str | None:
    txt = _strip_accents_lower(f"{name or ''} {description or ''}")
    hits = []
    for cat, needles in _CATEGORY_HINTS.items():
        if any(_strip_accents_lower(n) in txt for n in needles):
            hits.append(cat)
    if not hits:
        return None
    hits.sort(key=lambda c: _CATEGORY_ORDER_MAP.get(c, 10_000))
    return hits[0]


# Podutos Nomes

In [ ]:
def get_product_links_from_collection(max_pages: int = 8):
    links, vistos = [], set()
    for page in range(1, max_pages + 1):
        url = f"{COLLECTION_URL}?page={page}"
        try:
            r = requests.get(url, headers=HEADERS, timeout=30)
            if r.status_code != 200:
                break
            soup = BeautifulSoup(r.content, "html.parser")
            for a in soup.find_all("a", href=True):
                href = a["href"]
                if "/products/" in href:
                    full = urljoin(BASE_URL, href)
                    if full not in vistos:
                        vistos.add(full); links.append(full)
            time.sleep(0.7)
        except Exception as e:
            print(f"[get_product_links_from_collection] Erro na página {page}: {e}")
            break
    return links


def extract_ingredients(soup: BeautifulSoup):
    ingredientes_brutos = []
    area_ing = soup.find("div", class_=re.compile(r"\btabcontent\b.*\bingredients\b", re.I))
    if area_ing:
        for h2 in area_ing.find_all("h2"):
            txt = h2.get_text(" ", strip=True)
            if txt:
                ingredientes_brutos.append(txt)
        resume = area_ing.find("div", class_=re.compile(r"\bingredients_resume\b", re.I))
        if resume:
            raw = resume.get_text("\n", strip=True)
            raw_norm = _strip_accents_lower(raw)
            i1 = raw_norm.find("ingredientes:")
            if i1 != -1:
                bloco_raw = raw[i1 + len("ingredientes:") :]
                for stopper in ["ingredientes em portugues:", "ingredientes em português:"]:
                    cut_idx = _strip_accents_lower(bloco_raw).find(stopper)
                    if cut_idx != -1:
                        bloco_raw = bloco_raw[:cut_idx]
                        break
                ingredientes_brutos.extend(_tokenize_ingredientes(bloco_raw))
            for key in ["ingredientes em portugues:", "ingredientes em português:"]:
                j = raw_norm.find(key)
                if j != -1:
                    bloco_pt = raw[j + len(key) :]
                    ingredientes_brutos.extend(_tokenize_ingredientes(bloco_pt))
                    break

    tokens = []
    for item in ingredientes_brutos:
        if not item: continue
        t = item.strip().strip(":").strip()
        if not t: continue
        if len(t.split()) > 8 and "," not in t and ";" not in t:
            continue
        tokens.append(t)

    padronizados = _padroniza_por_lista(tokens, INGREDIENTES_VALIDOS)
    return "; ".join(padronizados)

## Tamanho

In [ ]:
def extract_size(soup: BeautifulSoup):
    size_element = soup.find("span", class_=re.compile(r"\bProductWeight\b", re.I))
    if size_element:
        return size_element.get_text(strip=True)
    for alt in [
        soup.find(class_=re.compile(r"weight", re.I)),
        soup.find(class_=re.compile(r"size", re.I)),
        soup.find(class_=re.compile(r"volume", re.I)),
        soup.find(class_=re.compile(r"quantity", re.I)),
    ]:
        if alt and alt.get_text(strip=True):
            text = alt.get_text(strip=True)
            m = re.search(r"\d+\,?\d*\s*(?:ml|g|mg|kg|l|oz)", text, re.I)
            if m: return m.group()
    return None

def extract_name(soup: BeautifulSoup):
    name_element = soup.find("span", id="ProductNameTitle")
    if name_element:
        return name_element.get_text(strip=True)
    for alt in [soup.find("h1"), soup.find("h2"), soup.find("title"),
                soup.find(class_=re.compile(r"product.*title", re.I)),
                soup.find(class_=re.compile(r"\bname\b", re.I))]:
        if alt and alt.get_text(strip=True):
            return alt.get_text(strip=True)
    return None


## Ingredientes
### Normaliza e padroniza usando a lista INGREDIENTES_VALIDOS.

In [ ]:

def _collect_collection_products_with_pagination(collection_url: str, max_pages: int = 20):
    encontrados = set()
    for page in range(1, max_pages + 1):
        page_url = _add_or_replace_page_param(collection_url, page)
        try:
            r = requests.get(page_url, headers=HEADERS, timeout=30)
            if r.status_code != 200:
                break
            soup = BeautifulSoup(r.content, "html.parser")
            anchors = soup.find_all("a", href=True)
            page_found = 0
            for a in anchors:
                href = a["href"]
                if "/products/" in href:
                    text = a.get_text(" ", strip=True) or ""
                    if not text:
                        img = a.find("img", alt=True)
                        if img and img.get("alt"):
                            text = img["alt"]
                    if text:
                        page_found += 1
                        encontrados.add(_strip_accents_lower(text))
            if page_found == 0:
                break
            time.sleep(0.5)
        except Exception as e:
            print(f"[skin-type-collect] Erro na página: {page_url} -> {e}")
            break
    return encontrados

def enrich_with_skin_types(products: list):
    if not products:
        return products

    # 1) Sementes por texto (sinônimos)
    for p in products:
        if not (p.get("tipo_pele") or "").strip():
            nome = p.get("nome", "")
            subt = p.get("subtitulo", "")
            bene = p.get("beneficios", "")
            initial = classify_skin_types_from_texts(nome, subt, bene)
            if initial:
                p["tipo_pele"] = "; ".join(initial)

    # 2) Enriquecimento por filtros (se SKIN_FILTER_URLS estiver preenchido)
    if SKIN_FILTER_URLS:
        print("\nColetando tipos de pele via filtros da Sallve...")
        name_to_idx = {}
        for idx, p in enumerate(products):
            n = _strip_accents_lower(p.get("nome", ""))
            if n:
                name_to_idx.setdefault(n, set()).add(idx)

        for canonical, url in SKIN_FILTER_URLS.items():
            print(f" - Filtro '{canonical}': {url}")
            norm_names = _collect_collection_products_with_pagination(url, max_pages=20)
            print(f"   · {len(norm_names)} nome(s) coletado(s)")
            for nn in norm_names:
                if nn in name_to_idx:
                    for idx in name_to_idx[nn]:
                        current = products[idx].get("tipo_pele", "") or ""
                        tipos = [t.strip() for t in current.split(";") if t.strip()]
                        if canonical not in tipos:
                            tipos.append(canonical)
                        if SKIN_TYPE_CANONICAL_ORDER:
                            order = {v: i for i, v in enumerate(SKIN_TYPE_CANONICAL_ORDER)}
                            tipos = sorted(tipos, key=lambda x: order.get(x, 999))
                        products[idx]["tipo_pele"] = "; ".join(tipos)
            time.sleep(0.6)

    # 3) default se ainda vazio
    for p in products:
        if not (p.get("tipo_pele") or "").strip():
            p["tipo_pele"] = "todos os tipos"

    return products

## Benefícios 
### Recebe lista, normaliza, filtra, padroniza

In [ ]:

def padroniza_beneficios(textos_beneficios):
    if not textos_beneficios: return []
    encontrados = set()
    norm_syn = {
        canonico: [_strip_accents_lower(s) for s in patt_list if s]
        for canonico, patt_list in BENEFIT_SYNONYMS_PT.items()
    }
    for txt in textos_beneficios:
        n = _strip_accents_lower(txt)
        for canonico, padds in norm_syn.items():
            if any(patt in n for patt in padds):
                encontrados.add(canonico)

    if BENEFIT_CANONICAL_ORDER:
        order_map = {name: i for i, name in enumerate(BENEFIT_CANONICAL_ORDER)}
        return sorted(list(encontrados), key=lambda x: order_map.get(x, 999))
    return sorted(list(encontrados))

def extract_beneficios(soup: BeautifulSoup):
    candidatos = []
    for det in soup.find_all("details", class_=re.compile(r"\bDifferentials\b", re.I)):
        for li in det.find_all("li"):
            txt = li.get_text(" ", strip=True)
            if not txt: continue
            tnorm = _strip_accents_lower(txt)
            if re.fullmatch(r"\d+(?:[.,]\d+)?", tnorm):  # evita pontuações/numéricos
                continue
            if any(w in tnorm for w in ["ponto","pontos","minha sallve"]):
                continue
            if len(txt) <= 200:
                candidatos.append(txt)

    if not candidatos:
        main = soup.find("article", class_=re.compile(r"\bRegularMain__content\b"))
        cont = main or soup
        for li in cont.find_all("li"):
            txt = li.get_text(" ", strip=True)
            if txt and len(txt) <= 160:
                candidatos.append(txt)
    cats = padroniza_beneficios(candidatos)
    return "; ".join(cats)

def _pick_best_from_srcset(srcset: str) -> str:
    best_url, best_w = None, -1
    for part in srcset.split(","):
        part = part.strip()
        if not part: continue
        m = re.match(r"(.+?)\s+(\d+)w", part)
        if m:
            url, w = m.group(1).strip(), int(m.group(2))
            if w > best_w:
                best_url, best_w = url, w
        else:
            if best_url is None:
                best_url = part
    return best_url


## Imagem

In [ ]:

def extract_image_url(soup: BeautifulSoup) -> str | None:
    img = soup.find("img", class_=re.compile(r"\bview__image\b", re.I))
    candidates = []
    if img:
        if img.get("srcset"): candidates.append(_pick_best_from_srcset(img["srcset"]))
        if img.get("src"):    candidates.append(img["src"])
    if not candidates:
        for im in soup.find_all("img"):
            classes = " ".join(im.get("class", []))
            if re.search(r"(product|image|gallery|media|hero|view)", classes, re.I):
                if im.get("srcset"):
                    candidates.append(_pick_best_from_srcset(im["srcset"]))
                if im.get("src"):
                    candidates.append(im.get("src"))
    if not candidates:
        meta = soup.find("meta", property="og:image")
        if meta and meta.get("content"):
            candidates.append(meta["content"])
    for url in candidates:
        if not url: continue
        url = _safe_join_url(url.strip())
        return urljoin(BASE_URL, url)
    return None

def _infer_ext_from_url(url: str) -> str:
    path = urlparse(url).path
    ext = os.path.splitext(path)[1].lower()
    if ext in [".jpg",".jpeg",".png",".webp",".gif"]:
        return ".jpg" if ext == ".jpeg" else ext
    return ".jpg"

def download_image(image_url: str, product_name: str) -> str | None:
    if not image_url: return None
    try:
        r = requests.get(image_url, headers=HEADERS, timeout=40)
        r.raise_for_status()
        ext = _infer_ext_from_url(image_url)
        base = _sanitize_filename(product_name)
        filename = f"{base}{ext}"
        dest = IMAGES_DIR / filename
        counter = 1
        while dest.exists():
            filename = f"{base}-{counter}{ext}"
            dest = IMAGES_DIR / filename
            counter += 1
        with open(dest, "wb") as f:
            f.write(r.content)
        return filename
    except Exception as e:
        print(f"[download_image] Falha ao baixar {image_url}: {e}")
        return None


## Preço

In [ ]:
def extract_price(soup: BeautifulSoup) -> str | None:
    for cls_pat in [r"\bTotalPrice\b", r"\bTotalPrice__CTA\b"]:
        el = soup.find(["strong","span"], class_=re.compile(cls_pat, re.I))
        if el and el.get_text(strip=True):
            p = _normalize_price_text(el.get_text(" ", strip=True))
            if p: return p

    box = soup.find(class_=re.compile(r"\bProductPrice\b", re.I))
    if box:
        p = _normalize_price_text(box.get_text(" ", strip=True))
        if p: return p
        strong = box.find("strong")
        if strong:
            p = _normalize_price_text(strong.get_text(" ", strip=True))
            if p: return p

    generic = soup.find(class_=re.compile(r"price", re.I))
    if generic:
        p = _normalize_price_text(generic.get_text(" ", strip=True))
        if p: return p

    return _normalize_price_text(soup.get_text(" ", strip=True))


## Categoria

In [ ]:
def extract_product_data_from_soup(soup: BeautifulSoup, product_url: str, nome: str):
    categoria = classify_category_from_name(nome)

    beneficios_txt = extract_beneficios(soup) or ""
    subtitulo_txt  = None  # ajuste se tiver seletor específico
    tipos_detectados = classify_skin_types_from_texts(nome, subtitulo_txt or "", beneficios_txt)

    data = {
        "marca": "sallve",
        "nome": nome,
        "categoria": categoria,
        "subtitulo": subtitulo_txt,
        "tamanho_quantidade": extract_size(soup),
        "preco": extract_price(soup),
        "ingredientes": extract_ingredients(soup),
        "beneficios": beneficios_txt,
        "tipo_pele": "; ".join(tipos_detectados) if tipos_detectados else "",
        "imagem": "",
        "url": product_url,
    }

    img_url = extract_image_url(soup)
    if img_url:
        img_filename = download_image(img_url, nome)
        if img_filename:
            data["imagem"] = img_filename
    return data


In [ ]:

def fetch_soup(url: str) -> BeautifulSoup | None:
    try:
        r = requests.get(url, headers=HEADERS, timeout=30)
        if r.status_code != 200:
            return None
        return BeautifulSoup(r.content, "html.parser")
    except Exception as e:
        print(f"[fetch_soup] Erro em {url}: {e}")
        return None

def scrape_sallve_products():
    print("Iniciando webscraping da Sallve...")
    print(f"Coletando produtos em {COLLECTION_URL} com paginação...")
    product_links = get_product_links_from_collection(max_pages=8)
    if not product_links:
        print("Nenhum produto encontrado na coleção. (Checar HTML/seletores)")
        return []

    print(f"Encontrados {len(product_links)} links de produto (antes de filtro).")
    products = []

    for i, url in enumerate(product_links, 1):
        print(f" [{i}/{len(product_links)}] {url}")
        soup = fetch_soup(url)
        if not soup:
            print("   ⚠️  Falha ao abrir página.")
            continue

        nome = extract_name(soup)
        if not nome:
            print("   ⚠️  Nome não encontrado.")
            continue

        if should_exclude_product(nome):
            print(f"   ❌ Excluído por keyword (models): {nome}")
            continue

        prod = extract_product_data_from_soup(soup, url, nome)
        products.append(prod)
        print(f"   ✅ OK: {nome}")
        time.sleep(0.7)

    print(f"Total pós-filtro: {len(products)}")
    return products


## Nome

## Requisição -> html do produto-> extrai dados -> busca pelos links

## Tipos de pele

In [ ]:
_SKIN_SYNONYMS_NORM = {
    canon: [_strip_accents_lower(x) for x in lst if x]
    for canon, lst in SKIN_TYPE_SYNONYMS_PT.items()
}
_SKIN_ORDER_MAP = {name: i for i, name in enumerate(SKIN_TYPE_CANONICAL_ORDER or [])}

def classify_skin_types_from_texts(*strings: str) -> list[str]:
    big = _strip_accents_lower(" ".join(s for s in strings if s))
    found = set()
    for canon, pats in _SKIN_SYNONYMS_NORM.items():
        if any(p and p in big for p in pats):
            found.add(canon)
    return sorted(found, key=lambda x: _SKIN_ORDER_MAP.get(x, 10_000))

## Arquivos

In [ ]:
def save_data(products_data):
    if not products_data:
        print("Nenhum dado para salvar.")
        return

    clean = []
    for p in products_data:
        clean.append({
            "marca": p.get("marca"),
            "nome": p.get("nome"),
            "categoria": p.get("categoria"),
            "subtitulo": p.get("subtitulo"),
            "tamanho_quantidade": p.get("tamanho_quantidade"),
            "preco": p.get("preco"),
            "ingredientes": p.get("ingredientes"),
            "beneficios": p.get("beneficios"),
            "tipo_pele": p.get("tipo_pele"),
            "imagem": p.get("imagem"),
            "url": p.get("url"),
        })

    with open("sallve_products.json", "w", encoding="utf-8") as f:
        json.dump(clean, f, ensure_ascii=False, indent=2)

    df = pd.DataFrame(clean)
    df.to_csv("sallve_products.csv", index=False, encoding="utf-8")

    print("\nDados salvos:")
    print(f"  - JSON: sallve_products.json ({len(clean)} produtos)")
    print(f"  - CSV:  sallve_products.csv  ({len(clean)} produtos)")

# ========= MAIN =========
if __name__ == "__main__":
    try:
        data = scrape_sallve_products()
        data = enrich_with_skin_types(data)  # usa sinônimos + (opcional) filtros
        save_data(data)
        print(f"\nConcluído! Produtos extraídos: {len(data)}")
        print(f"Imagens salvas em: {IMAGES_DIR.resolve()}")
    except Exception as e:
        print(f"\nERRO: {e}")


Iniciando webscraping da Sallve...
Coletando produtos em /collections/loja com paginação...
Encontrados 138 links de produto (antes de filtro).
 [1/138] https://www.sallve.com.br/products/bastao-antioxidante-para-olhos

ERRO: 'list' object has no attribute 'items'
